# Introduction

The algorithm is broken down into multiple created functions, where the following order counts: <br /> <br /> 
$\;\;\;\;\;\;\;\;\;\;\;\;\;$ 1) $\;\;\;\;\;$Convert matrix into a system of coordinates and remove the wall coordinates <br /> 
$\;\;\;\;\;\;\;\;\;\;\;\;\;$ 2) $\;\;\;\;\;$Cluster the neighbouring coordinates (akin to detecting local areas) <br /> 
$\;\;\;\;\;\;\;\;\;\;\;\;\;$ 3) $\;\;\;\;\;$Detect the number of required cleaning sessions <br /> 

# 1) Convert matrix into a system of coordinates and remove the wall coordinates

### $\;\;\;\;\;\;\;\;\;\;\;$Detect whether the coordinate corresponds to a wall

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$The inputs are: <br />
$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ $-$ The matrix X <br />
$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ $-$ The i (= row) coordinate <br />
$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ $-$ The j (= column) coordinate

In [6]:
def is_wall(X, i, j):
    single_cell = X[i][j]
                    
    # If wall, return True
    if single_cell == '#':
        return True
        
    else:
        return False

#### $\;\;\;\;\;\;\;\;\;\;\;$ Convert the matrix into list of coordinates (i = row, j = column)

$\;\;\;\;\;\;\;\;\;\;\;\;\;$ Each position in the matrix has an associated coordinate, with the first cell on the top left being coordinate (0, 0).  <br /> 
$\;\;\;\;\;\;\;\;\;\;\;\;\;$ The bottom right cell is coordinate (n-1, n-1), where n is the length of the row/columns (1 is substracted from n since the index in Python starts at 0 and not 1).  <br />$\;\;\;\;\;\;\;\;\;\;\;\;\;$ For simplicity, the assumtion is made that the number of rows equals the numer of columns:<br /><br />   <big>$$ n_{i} = n_{j} $$</big>  <br />

$\;\;\;\;\;\;\;\;\;\;\;\;\;$ We are only interested in neighbouring rooms and using this method, it is not necessary to keep the coordinates of the walls. Therefore, in case the coordinate corresponds to a wall in the matrix, it is excluded.

In [8]:
def clean_coordinates(X):

    n_rows = len(X)
    n_cols = len(X[0])  # The assumtion is made that the length is equal for each row
                    
    coordinates = []
        
    for i in range(n_rows):
        for j in range(n_cols):
            # If wall, continue to the next coordinate
            if X[i][j] == "#":
                continue
            else:
                coordinates.append((i, j))
 
    return coordinates

# 2) Cluster the neighbouring coordinates (akin to detecting local areas)

### $\;\;\;\;\;\;\;\;\;\;\;$ Detect whether coordinate x neighbours coordinate y

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$By taking the absolute difference of two different coordinates, we can verify whether they are neighbors. The total absolute difference should equal 1, as we only consider horizontal or vertical neighbors (ignoring diagonal neighbours, $\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$which would require a total absolute difference of 2).

$$ \text{Coordinate 1} = (i_{1}, j_{1}) \quad \text{and} \quad \text{Coordinate 2} = (i_{2}, j_{2}) $$

$$ \text{Then, the absolute difference between the coordinates is:} $$ 

$$ \text{Absolute Difference} = \left| i_{1} - i_{2} \right| + \left| j_{1} - j_{2} \right| $$

$$ \text{If } \left| i_{1} - i_{2} \right| + \left| j_{1} - j_{2} \right| = 1, \text{ then } (i_{1}, j_{1}) \text{ and } (i_{2}, j_{2}) \text{ are neighbouring coordinates.} $$


In [10]:
def detect_neighbour(cor, cor2):
    
    i, j = cor
    i2, j2 = cor2
    absolute_diff = abs(i - i2) + abs(j - j2)

    # Neighouring scenario
    if absolute_diff == 1:
        return True
    # Not neighbouring scenario
    else:
        return False

### $\;\;\;\;\;\;\;\;\;\; \;$ Find one local area

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ To find a single local area, the following code is used. It iterates over the coordinates given as input, and clusters neighbouring coordinates together. This process is repeated until no neighbouring coordinates are found. 

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ To find the first search, the following code is used:

In [12]:
def find_local_area(X):
    
    # Pre-see with the first coordinate
    local_area = [X[0]]
    # This indicates if new neighbors were added. Initialize to true to start off the while loop
    neighbouring =  True 
    
    while neighbouring:
        # Change back to False, and alter if indeed a neighbour is detected
        neighbouring = False 
        
        for cor in X[1:]:
            for cor2 in local_area:
                if detect_neighbour(cor, cor2) and cor not in local_area:
                    local_area.append(cor)
                    # Set back to True in case a neighbouring coordinate is found
                    neighbouring = True
                    break

    return local_area

### $\;\;\;\;\;\;\;\;\;\; \;$ Find all local areas

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ In the below, local areas are retrieved until all are found

In [14]:
def find_all_local_areas(X):

    # Import a flat (one dim.) list of coordinates excluding walls
    clean_cors = clean_coordinates(X)
    local_areas = []

    # The while loop stops when all local areas are found
    while clean_cors:
        
        local_area = find_local_area(clean_cors)
        local_areas.append(local_area)
        
        for cor in local_area:
            clean_cors.remove(cor)
    
    return local_areas    

# 3) Detect the number of required cleaning sessions

### $\;\;\;\;\;\;\;\;\;\;\;$ Detect whether there is a dirty room in the local areas

In [16]:
def is_dirty(i, j, X):
    single_position = X[i][j]
    
    if single_position == '1':
        return True
    
    else:
        return False

### $\;\;\;\;\;\;\;\;\;\;\;$ Iterate over local areas and count the required clean sessions

$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ Iterate over the rooms in each local area. If a local area contains at least one dirty room, add 1 to the number of required cleaning sessions and continue to the next local area

In [18]:
def n_cleans_required(X):
    
    local_areas = find_all_local_areas(X)
    n_cleans = 0
    
    # Iterate over each local area
    for local_area in local_areas:
        dirty_room = False
        
        # Iterate over the rooms in the lcoal area
        for rooms_cor in local_area: 
            i, j = rooms_cor
            
            '''In case there is a dirty room in a local alrea, add 
                one cleaning session and go to the next local area 
                (by breaking the inner loop)'''
            if is_dirty(i, j, X):
                dirty_room = True
                n_cleans += 1
                break
        
    return 'Number of local areas: ', len(local_areas), '\nNumber of cleans required: ', n_cleans